# 2.3 Special Columns (Ascending Keys)


As per [https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-tables-statistics](https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-tables-statistics)

_Consider updating "ascending key" columns such as transaction dates more frequently because these values won't be included in the statistics histogram._

As a demo lets show a table with a date missing and perfect stats. Also introducing the StatsOptimize SProc to automate updating of Stats

In [14]:
delete from  [dbo].[FactFinance100m_nostats] WHERE DateKey> 20130000

exec  dbo.[StatsOptimize]  @Tables='FactFinance100m_nostats'
, @StatisticsModificationLevel=0
, @StatisticsSample=100 
, @OnlyModifiedStatistics=null
, @DeleteOverlappingStats=null
, @TimeLimit=null 
, @Execute='Y'


SELECT * FROM vStats WHERE table_name ='FactFinance100m_nostats'


(10000000 rows affected)

Date and time: 2022-07-09 09:29:24

Server: aw-dev

Version: 10.0.15665.0

Edition: SQL Azure

...

SqlCommand: UPDATE STATISTICS [dbo].[FactFinance100m_nostats] WITH FULLSCAN

Date and time: 2022-07-09 09:29:33

(3 rows affected)

Total execution time: 00:00:17.980

object_id,stat_name,table_name,schema_name,stats_id,auto_created,filter_definition,last_updated_date,stat_columns,stats_row_count,actual_row_count,stats_difference_percent,stats_sample_rate,sqlCommand,user_created
354868381,ClusteredIndex_790bfcda58a449a7a8517a169f0e7e0a,FactFinance100m_nostats,dbo,1,0,NULL,NULL,"DateKey,DepartmentGroupKey,ScenarioKey,OrganizationKey,AccountKey,Amount,Date,LineageKey",67403926,67403926,0.00,12,UPDATE STATISTICS [dbo].[FactFinance100m_nostats] (ClusteredIndex_790bfcda58a449a7a8517a169f0e7e0a) WITH SAMPLE 12 PERCENT,0
354868381,_WA_Sys_00000001_1526DC9D,FactFinance100m_nostats,dbo,2,1,NULL,2022-07-09 09:29:32.500,DateKey,67403926,67403926,0.00,12,UPDATE STATISTICS [dbo].[FactFinance100m_nostats] (_WA_Sys_00000001_1526DC9D) WITH SAMPLE 12 PERCENT,0
354868381,_WA_Sys_00000005_1526DC9D,FactFinance100m_nostats,dbo,3,1,NULL,2022-07-09 09:29:32.690,AccountKey,67403926,67403926,0.00,12,UPDATE STATISTICS [dbo].[FactFinance100m_nostats] (_WA_Sys_00000005_1526DC9D) WITH SAMPLE 12 PERCENT,0


In [15]:
/* The Stats Histogram is now mising latest date (while being perfect)*/
DBCC SHOW_STATISTICS ('FactFinance100m_nostats',_WA_Sys_00000001_1526DC9D)

(1 row affected)

(1 row affected)

(27 rows affected)

Total execution time: 00:00:00.080

Name,Updated,Rows,Rows Sampled,Steps,Density,Average key length,String Index,Filter Expression,Unfiltered Rows,Persisted Sample Percent
_WA_Sys_00000001_1526DC9D,Jul 9 2022 9:29AM,67403926,67403926,27,0.037037037,4,NO,NULL,67403926,0


All density,Average Length,Columns
0.037059914,4,DateKey


RANGE_HI_KEY,RANGE_ROWS,EQ_ROWS,DISTINCT_RANGE_ROWS,AVG_RANGE_ROWS
20101229,0,4318566,0,1
20110129,0,4364350,0,1
20110301,0,4357974,0,1
20110331,0,3807165,0,1
20110501,0,3409494,0,1
20110531,0,3667331,0,1
20110701,0,3336511,0,1
20110801,0,3064507,0,1
20110829,0,3118080,0,1
20110929,0,2602791,0,1


In [16]:
/* Lets insert 10 million rows into today to simulate a load */
INSERT INTO FactFinance100m_nostats ([AccountKey], [ScenarioKey], [DepartmentGroupKey], [DateKey], [OrganizationKey], [Amount], [Date], [LineageKey])
SELECT  TOP 10000000 [AccountKey], [ScenarioKey], [DepartmentGroupKey], [DateKey], [OrganizationKey], [Amount], [Date], [LineageKey]  
FROM [dbo].[FactFinance1b]  WHERE dateKey >20130000

(10000000 rows affected)

Total execution time: 00:00:16.364

In [17]:
/* Lets Upodate any stat that is based on DateKey */
exec  dbo.[StatsOptimize]  @Tables='%.%.DateKey'
, @StatisticsModificationLevel=null
, @StatisticsSample=100 
, @OnlyModifiedStatistics=null
, @DeleteOverlappingStats=null
, @TimeLimit=null 
, @Execute='Y'

/* The Stats Histogram is now perfect */
DBCC SHOW_STATISTICS ('FactFinance100m_nostats',_WA_Sys_00000001_1526DC9D)

Date and time: 2022-07-09 09:30:32

Server: aw-dev

Version: 10.0.15665.0

Edition: SQL Azure

Parameters: @Tables= '%.%.DateKey', @OnlyModifiedStatistics = 'Y', @StatisticsModificationLevel = NULL, @StatisticsSample = 100, @DeleteOverlappingStats = N, @TimeLimit = NULL, @Execute = Y

...

(39 rows affected)

Total execution time: 00:00:09.373

Name,Updated,Rows,Rows Sampled,Steps,Density,Average key length,String Index,Filter Expression,Unfiltered Rows,Persisted Sample Percent
_WA_Sys_00000001_1526DC9D,Jul 9 2022 9:30AM,77403926,77403926,39,0.025641026,4,NO,NULL,77403926,0


All density,Average Length,Columns
0.025740026,4,DateKey


RANGE_HI_KEY,RANGE_ROWS,EQ_ROWS,DISTINCT_RANGE_ROWS,AVG_RANGE_ROWS
20101229,0,4318566,0,1
20110129,0,4364350,0,1
20110301,0,4357974,0,1
20110331,0,3807165,0,1
20110501,0,3409494,0,1
20110531,0,3667331,0,1
20110701,0,3336511,0,1
20110801,0,3064507,0,1
20110829,0,3118080,0,1
20110929,0,2602791,0,1
